<a href="https://colab.research.google.com/github/shaivi-yarlagadda/BNFO301_Yarlagadda_Shaivi/blob/main/BNFO301_2024_Lab5_2_student_wEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>BNFO301 Lab Assignment 5.2: Consensus Sequences</h1>

A consensus sequence represents the most frequent nucleotide at each position in an alignment of two or more sequences. You can think of this as finding the most common base in a position within a multiple sequence alignment. Consensus sequences can be useful for identifying and visualizing motifs in a DNA or amino acid sequence. You have been provided a FASTA file of aligned sequences. The goal of this assignment is to write a python script to read in the sequences from the file and generate the consensus sequence


Grading: problem 3 = 10 points.   ExtraCredit = 3 points.

Timeline: You will have two class periods to complete the lab.    Your responses to problems 1-3 should be uploaded to **github**.  Note, answers to problems one and two, from last meeting, are given.

**Helpful Resources:**

If you are unfamiliar with consensus sequences or would like to review, this video explains the concept: https://www.youtube.com/watch?v=4HYJILahPw4 between 2:20 and 4:20. It may also be helpful to first generate the consensus sequence by hand, so that you can check the output from your script.

**Reminder: You may work with other students in the class, but you are not allowed to copy code from others or any source.  If you copy anything from any source, you must describe the source.  Failure to comply is a violation of class policy.**



### Setup
Load the Sequence Data File.  Please run this block without changing the code.

This file is in fasta format and contains an alignment of DNA sequences, including gaps

you can view the file by clicking on this link: https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/Nucl_alignment.fa

In [1]:
import os.path
# Load the genbank file
DATA_FILE_GITHUB = "https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/Nucl_alignment.fa"
DEFAULT_FILE_NAME = 'Nucl_alignment.fa'

fileName = DEFAULT_FILE_NAME
#Does the file exists locally, if not get it from the github
if not os.path.exists(fileName):
  #Load the file from Github to the local folder
  !wget --no-check-certificate --content-disposition $DATA_FILE_GITHUB



--2024-02-22 17:32:33--  https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/Nucl_alignment.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302 [text/plain]
Saving to: ‘Nucl_alignment.fa’

Nucl_alignment.fa   100%[===================>]     302  --.-KB/s    in 0s      

2024-02-22 17:32:33 (27.6 MB/s) - ‘Nucl_alignment.fa’ saved [302/302]



###problem-1.  Create a definition to read the FASTA formated file and add sequences into a list.  Create a second list that contains your headers (with the ">" removed).  Your function should return both lists, not just print each list.  You have been provided a fucntion to print each lise, so that you can evaluate your lists.


Hint, you will need to do the following:
* We need a function to load the sequence file
* We will load it in two lists of sequences and header names
* Remember to strip ">" and "\n" characters


Your output should look like this:

Human       GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT

Gorilla     GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT

Dolphin     GAGGCTC---GGAGCCGGACCTGGACCCCTGCGATAGCCGTCTG-CTCCCG

Rat         GGAGCAACTAGGAACCCGAACCAGAGCCCGGCGAGCGCAGCCTGCAGCTCC

Mouse       GAGGCGCCTAGGAACCCGAGCCGGAGCTCAGCGAGCGCAGCCTGCAGCTCC


In [2]:
#ANSWER##################################

def ReadFile(fh):
    # Open file
    file = open(fh)

    # Initialize variables
    sequences = []  # Store sequences
    seqIds = []  # Store sequence headers

    # Loop through file
    for line in open(fh):

        # Remove all unwanted white spaces
        line = line.strip()

        # Find fasta header for first sequence
        if line.startswith(">"):
            seqIds.append(line.replace(">", ""))
        else:
            sequences.append(line)

    # Return lists
    return sequences, seqIds


# -----------------------------
# Utility function to pretty print
# ------------------------------
def printInfo(seqIds, seqs):
    for seqId, seq in zip(seqIds, seqs):
        print("{: <12} {: <20}".format(seqId, seq))


sequences, seqIds = ReadFile(fileName)
# print(sequences, seqIds)
printInfo(seqIds, sequences)




 Human       GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT
 Gorilla     GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT
 Dolphin     GAGGCTC---GGAGCCGGACCTGGACCCCTGCGATAGCCGTCTG-CTCCCG
 Rat         GGAGCAACTAGGAACCCGAACCAGAGCCCGGCGAGCGCAGCCTGCAGCTCC
 Mouse       GAGGCGCCTAGGAACCCGAGCCGGAGCTCAGCGAGCGCAGCCTGCAGCTCC


###problem-2. In this problem you will build a defition that uses nested loops to format the data in such a way that you can determine the most common base.  Multiple bioinformatic programs handle data in this way.  Remember to return your data as strings that represent bases in a column.  You have been given code to print the first three columns of the matrix.  This allows you to visualize your matrix and check it for accuracy.  Functions len and range will be useful in completing this problem.


For example, lets take the input for the first three bases in the alignment given:

Human       GGA

Gorilla     GGA

Dolphin     GAG

Rat         GGA

Mouse       GAG

You should return data that looks like this:  [['G', 'G', 'G', 'G', 'G'], ['G', 'G', 'A', 'G', 'A'], ['A', 'A', 'G', 'A', 'G']]

In [32]:
#ANSWER###################################

# -----------------------------
# Convert sequence to matrix for easy computation
# ------------------------------
def generateMatrix(seqs):
    # Lengths of the aligned sequences are equal
    # Get length of first sequence
    length = len(seqs[0])

    # Initialize variables
    matrix = []  # Will store all values at each position

    # Get a list of list
    # Reads fasta file as matrix
    # Example:
    #        ATGCA
    #        ATGAA
    #        TCGAT
    #             Bases at index 0   Bases at index 1 ...
    # positions = [["A", "A", "T"],  ["T", "T", "C"], ...]]

    for base in range(length):
        column = []
        for sequence in range(len(seqs)):
            column.append(seqs[sequence][base])
        matrix.append(column)

    return matrix


matrix = generateMatrix(sequences)

#printing first 3 rows to show the matrix
print(matrix[0:3])





[['G', 'G', 'G', 'G', 'G'], ['G', 'G', 'A', 'G', 'A'], ['A', 'A', 'G', 'A', 'G']]


###problem-3.  In this last problem, you will create a defintion that returns a consensus sequence with the most common base. You will again need to use nested loops.  Functions get and max will be helpful in responding to this question.  

Use must use the following rules:  This function should print the consensus sequence as one continuous string.

1. If all bases are in agreement, return base as upper.case.  e.g. ['G', 'G', 'G', 'G', 'G'] returns G
2. If there is an equal split between bases in a column, return "n".  e.g. ['G', 'G', 'A', 'A', 'A', 'G'] returns n
3. If the the bases are not in agreement, but one base is more frequent than other, return the most frequent base in lower case.  e.g. ['G', 'G', 'G', 'G', 'A'] returns g
4. Consider gap characters when constructing your consensus sequence. e.g. ['G', 'G', '-', 'G', 'G'] returns g

Your output should look like this: GgaGcgnctnGGAgCCgGacCcgGAcCcCgGCGAtnGCcGcCtGcntCtCn


In [36]:
# -----------------------------
# create a function to calculate the consensus
# ------------------------------
def getConsensus(NewlyFormatedData):
  #do not forget to initialize variable in which you can store your concensus string
  consensus = ""
  counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0, '-':0}

  for i in range(len(NewlyFormatedData[0])):
    for sequence in NewlyFormatedData:
      if sequence[i] in counts:
        counts[sequence[i]] += 1

    #total_count = total(counts.values())
    total_count = len(NewlyFormatedData[0])
    if total_count == sum(counts.values()):
      consensus += total(counts, key=counts.get).upper()
    elif total_count * 2 > sum(counts.values()):
      consensus += total(counts, key=counts.get).lower()
    else:
      consensus += 'n'

  #remember to return your consensus
  return consensus

#do not forget to call your function
consensus = getConsensus(matrix)

#finally, print the consensus sequence as one string
print(consensus)

nnnnn


###ExtraCredit:

Create a defintion that returns a consensus sequence, just as you did in response to problem #3.  However, in this case, your definition should ignore gap characters.  For example, ['G', 'G', '-', 'G', 'G'] returns G